# HTTP Server for Image Upload & Get Classification file 

Http server listening on port **5000** for upload files or get json classification content:

* http://[hostname]:5000
to POST an image file.
* http://[hostname]:5000/inference?filename=[filesent]
to get JSON file holding classification, where: [*filesent*] is same file name sent with previous POST. If no classification hasn't been done it returns string "NOT_FOUND"

server create a ".ck" version for filesent to avoid conflict on get.

In [ ]:
import os
from flask import Flask, request, redirect, url_for,Response
from werkzeug.utils import secure_filename

UPLOAD_FOLDER = 'static/img'
ALLOWED_EXTENSIONS = set(['txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'])

#app = Flask(__name__)
app = Flask(os.environ["SAMPLE_MAINTENANCE"]+'server')

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def root_dir():  # pragma: no cover
    #print('TEST')
    #return os.path.abspath(os.path.dirname(__file__))
    return os.environ["SAMPLE_MAINTENANCE"]+'server'


def get_file(filename):  # pragma: no cover
    try:

        src = os.path.join(root_dir(), filename)
        # Figure out how flask returns static files
        # Tried:
        # - render_template
        # - send_file
        # This should not be so non-obvious
        return open(src).read()
    except IOError as exc:
        return str("NOT_FOUND")

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        print(root_dir())
        # check if the post request has the file part
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        # if user does not select file, browser also
        # submit a empty part without filename
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            open(os.path.join(app.config['UPLOAD_FOLDER'], filename+'.ck'),'x').close()
            return redirect(url_for('upload_file',filename=filename))

    return '''
    <!doctype html>
    <title>Upload new File</title>
    <h1>Upload new File</h1>
    <form method=post enctype=multipart/form-data>
      <p><input type=file name=file>
         <input type=submit value=Upload>
    </form>
    getfile: http://[hostname]:5000/inference?filename=filesent
    '''

@app.route('/inference', methods=['GET'])
def metrics():  # pragma: no cover
    fileResponse=request.args.get('filename', '')
    content = get_file(UPLOAD_FOLDER+"/"+fileResponse+".json")
    print(UPLOAD_FOLDER+"/"+fileResponse+".json")
    return Response(content, mimetype="application/json")
if __name__ == '__main__':
    print(root_dir())
    #app.run(debug=True)
    app.run(host='0.0.0.0')